In [2]:
%matplotlib inline

import pandas as pd 
import numpy as np 
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib
import matplotlib.pyplot as plt 
import math
from sklearn import linear_model, cross_validation, metrics
from sklearn import feature_selection as fs
from sklearn import preprocessing

df = pd.read_csv('../data/cleanedSeasonStats.csv')
for col in df.columns:
    if 'Unnamed' in col or col[-1] == '-': # meaningless columns
        del df[col]

# Scikit breaks with NaNs, replace with 0s
df.fillna(0, inplace=True)
for col in df.columns:
    print col

year
player
totals-Rk
totals-Pos
totals-Age
totals-Tm
totals-G
totals-GS
totals-MP
totals-FG
totals-FGA
totals-FG%
totals-3P
totals-3PA
totals-3P%
totals-2P
totals-2PA
totals-2P%
totals-eFG%
totals-FT
totals-FTA
totals-FT%
totals-ORB
totals-DRB
totals-TRB
totals-AST
totals-STL
totals-BLK
totals-TOV
totals-PF
totals-PTS
per_game-Rk
per_game-Pos
per_game-Age
per_game-Tm
per_game-G
per_game-GS
per_game-MP
per_game-FG
per_game-FGA
per_game-FG%
per_game-3P
per_game-3PA
per_game-3P%
per_game-2P
per_game-2PA
per_game-2P%
per_game-eFG%
per_game-FT
per_game-FTA
per_game-FT%
per_game-ORB
per_game-DRB
per_game-TRB
per_game-AST
per_game-STL
per_game-BLK
per_game-TOV
per_game-PF
per_game-PTS
per_minute-Rk
per_minute-Pos
per_minute-Age
per_minute-Tm
per_minute-G
per_minute-GS
per_minute-MP
per_minute-FG
per_minute-FGA
per_minute-FG%
per_minute-3P
per_minute-3PA
per_minute-3P%
per_minute-2P
per_minute-2PA
per_minute-2P%
per_minute-FT
per_minute-FTA
per_minute-FT%
per_minute-ORB
per_minute-DRB
per_min

/Users/james/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
df.head()

,year,player,totals-Rk,totals-Pos,totals-Age,totals-Tm,totals-G,totals-GS,totals-MP,totals-FG,...,advanced-OWS,advanced-DWS,advanced-WS,advanced-WS/48,advanced-OBPM,advanced-DBPM,advanced-BPM,advanced-VORP,curr-eff,next-eff
0,1985,Kareem Abdul-Jabbar,1,C,37,LAL,79,79,2630,723,...,7.6,3.6,11.2,0.204,3.6,1.3,4.9,4.6,26.1,24.2
1,1985,Alvan Adams,2,PF,30,PHO,82,69,2136,476,...,3.5,3.3,6.8,0.152,2.3,2.1,4.4,3.5,18.3,15.5
2,1985,Mark Aguirre,3,SF,25,DAL,80,79,2699,794,...,5.3,1.9,7.2,0.128,3.6,-1.9,1.7,2.5,21.2,20.1
3,1985,Danny Ainge,4,SG,25,BOS,75,73,2564,419,...,3.9,2.8,6.6,0.124,1.3,0.4,1.7,2.4,16.3,13.8
4,1985,Ron Anderson,7,SF,26,CLE,36,7,520,84,...,0.0,0.3,0.3,0.032,-2.4,-1.0,-3.5,-0.2,5.3,9.7


In [4]:
# Specify the features that we want to use.

# For now, use everything numerical (except for obviously next_eff)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
feature_cols = list(set(df.select_dtypes(include=numerics).columns.values) - {'next-eff'})

# Get X, and scale/normalize (do we want/need to scale?)
data = df[feature_cols]
data = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(data), columns=data.columns)

# Get y
target = df['next-eff']

# Normalize the data on just a few columns
# NOTE: This notebook doesn't use this, but svmrank might want it
cols_to_norm = ['per_game-PTS', 'per_game-TRB', 'per_game-AST', 'per_game-STL', 'per_game-BLK', 'per_game-FG', 'per_game-FT', 'per_game-FGA', 'per_game-FTA', 'per_game-TOV']
norm_df = df[['next-eff', 'year', 'player'] + cols_to_norm]
norm_df[cols_to_norm] = norm_df[cols_to_norm].apply(lambda x: (x / abs(x.max())))
norm_df.fillna(0, inplace=True)
norm_df.to_csv('cleanedSeasonStats0to1.csv', index=False)
norm_df.head()

/Users/james/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/Users/james/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,next-eff,year,player,per_game-PTS,per_game-TRB,per_game-AST,per_game-STL,per_game-BLK,per_game-FG,per_game-FT,per_game-FGA,per_game-FTA,per_game-TOV
0,24.2,1985,Kareem Abdul-Jabbar,0.592992,0.422460,0.220690,0.216216,0.375000,0.686567,0.362745,0.550360,0.381679,0.438596
1,15.5,1985,Alvan Adams,0.396226,0.326203,0.262069,0.378378,0.107143,0.432836,0.294118,0.402878,0.267176,0.421053
2,20.1,1985,Mark Aguirre,0.692722,0.320856,0.213793,0.216216,0.053571,0.738806,0.539216,0.705036,0.557252,0.561404
3,13.8,1985,Danny Ainge,0.347709,0.192513,0.365517,0.432432,0.017857,0.417910,0.156863,0.381295,0.137405,0.350877
4,9.7,1985,Ron Anderson,0.156334,0.128342,0.062069,0.081081,0.035714,0.171642,0.107843,0.194245,0.106870,0.157895


In [105]:
# Let's try Lasso, letting scikit pick alpha.
# lasso = linear_model.LinearRegression()
lasso = linear_model.LassoCV(max_iter=5000, normalize=True, cv=5)
lasso.fit(data, target)

# 5-fold Cross Validation
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(lasso, data, target, cv=kfold, scoring='r2').mean()

# It seems to do ~okay~, but we can do better. Lasso is not going to
# be super stable, because a lot of our features are going to be 
# very, very correlated. See this for other methods which I will try
# in the following cells. Furthermore, the features that we ~do~ get
# are impacted by lots of duplication across "games", "poss", etc. 

0.74062145195310247

In [6]:
# Get features columns that don't overlap 
def is_redundant(col):
    return 'totals' in col or 'per_game' in col or 'per_minute' in col

redundant = filter(lambda col: is_redundant(col), df.columns.values)
feature_cols = list(set(df.select_dtypes(include=numerics).columns.values) \
                    - {'next-eff'} \
                    - set(redundant))
data = df[feature_cols]

# Try Lasso again
lasso = linear_model.LassoCV(max_iter=5000, normalize=True, cv=5)
lasso.fit(data, target)

# 5-fold Cross Validation
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
print cross_validation.cross_val_score(lasso, data, target, cv=kfold, scoring='r2').mean()

# View coefficients
coefficients = sorted([(feature_cols[i], lasso.coef_[i]) for i in range(len(feature_cols))], key=lambda tup: abs(tup[1]), reverse=True)
filter(lambda tup: abs(tup[1]) != 0, coefficients)

0.739812715502


[('advanced-TS%', -5.5506973214834439),
 ('advanced-WS/48', 4.9572228346187508),
 ('curr-eff', 0.78651305785353842),
 ('advanced-FTr', 0.41030597283508174),
 ('per_poss-3P%', -0.24633381870640958),
 ('advanced-VORP', 0.24175114150472349),
 ('advanced-Age', -0.21599592250381766),
 ('per_poss-AST', 0.18911010252816066),
 ('per_poss-TOV', 0.18830254290790627),
 ('per_poss-2P%', 0.16927026580549354),
 ('advanced-DWS', 0.16487895059442961),
 ('per_poss-PF', -0.084285458766000645),
 ('advanced-WS', 0.079706919283602601),
 ('advanced-USG%', 0.07844199403898941),
 ('advanced-STL%', -0.066375609413050457),
 ('advanced-PER', -0.064190361589460426),
 ('advanced-AST%', -0.058099915434950286),
 ('per_poss-3PA', -0.051252935559142627),
 ('advanced-BLK%', 0.039288866133187189),
 ('advanced-OBPM', 0.037719301589134911),
 ('per_poss-DRtg', -0.03540009090037062),
 ('per_poss-ORB', -0.034082243282987104),
 ('per_poss-Age', -0.018300619269398213),
 ('per_poss-GS', -0.014869696467941469),
 ('advanced-TRB%'

In [7]:
# Okay, so Lasso isn't very stable, which is expected. 
# What is NOT expected is a fat negative weight with advanced_TS%. 
# Getting some lucky threes will hugely inflate that, which with
# squared error is no bueno.

# Try removing players that didn't play much that season.
nbw_df = df[df['totals-MP'] >= df['totals-MP'].quantile(.1)] # nbw, short for no benchwarmers
nbw_data = nbw_df[feature_cols]
nbw_target = nbw_df['next-eff']
nbw_lasso = linear_model.LassoCV(max_iter=5000, normalize=True, cv=5)
nbw_lasso.fit(nbw_data, nbw_target)

# 5-fold cross validation again
kfold = cross_validation.KFold(len(nbw_target), n_folds=5, shuffle=True)
print cross_validation.cross_val_score(nbw_lasso, nbw_data, nbw_target, cv=kfold, scoring='r2').mean()

# View coefficients
coefficients = sorted([(feature_cols[i], nbw_lasso.coef_[i]) for i in range(len(feature_cols))], key=lambda tup: abs(tup[1]), reverse=True)
filter(lambda tup: abs(tup[1]) != 0, coefficients)

# plt.scatter(nbw_df['advanced-WS/48'], nbw_target)
# plt.show()

0.734647816189


[('advanced-WS/48', 12.343328944056179),
 ('advanced-TS%', -6.5038230898405995),
 ('advanced-FTr', 0.90134042268242431),
 ('curr-eff', 0.73731497027060933),
 ('per_poss-FT%', 0.53565769759129167),
 ('per_poss-ORB', -0.34992253444233523),
 ('advanced-3PAr', 0.29287802463058527),
 ('per_poss-TOV', 0.25907178258359148),
 ('advanced-Age', -0.22421161557271815),
 ('per_poss-3P%', -0.20218177554862143),
 ('per_poss-PF', -0.18552810091564068),
 ('per_poss-AST', 0.18358849513928457),
 ('advanced-PER', -0.17562752117296146),
 ('advanced-OBPM', 0.14602489189253823),
 ('per_poss-BLK', 0.12909373970358018),
 ('advanced-VORP', 0.12273096862332343),
 ('advanced-ORB%', 0.10877662864323549),
 ('advanced-TRB%', 0.10060732822757563),
 ('per_poss-2PA', 0.096967141130980664),
 ('per_poss-STL', 0.095331974033196556),
 ('advanced-WS', 0.068164300707347747),
 ('advanced-USG%', 0.051587498385934023),
 ('advanced-AST%', -0.049413031437908345),
 ('advanced-BLK%', 0.040843417051196131),
 ('per_poss-DRtg', -0.037

In [8]:
# Removing people that hardly played didn't really benefit very much. 
# Let's fly through a bunch of models and try to see which performs 
# well without any feature engineering. 

In [9]:
# Elastic Net (use L1 and L2 error)
"""NOTE: I let it use 4 threads. Edit n_jobs if you want to limit that."""
elastic_net = linear_model.ElasticNetCV(n_jobs=4, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=50000, normalize=True, cv=5)
elastic_net.fit(data, target)
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(elastic_net, data, target, cv=kfold, scoring='r2').mean()
# plt.scatter(target, elastic_net.predict(data))
# plt.show()

0.74029035291667733

In [10]:
# RANSAC (robust regression, hopefully resists outliers)
ransac = linear_model.RANSACRegressor(max_trials=1000) # TODO: pick features better? (cross-validation?)
ransac.fit(data, target) 
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(ransac, data, target, cv=kfold, scoring='r2').mean()
# plt.scatter(target, elastic_net.predict(data))
# plt.show()

0.41216140227721809

In [11]:
# Boosted regression trees (adaboost and decision trees)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
regr_1 = DecisionTreeRegressor(max_depth=4) # TODO: pick features better? (cross-validation?)
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4)) # TODO: pick features better? (cross-validation?)
regr_1.fit(data, target)
regr_2.fit(data, target)
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(regr_2, data, target, cv=kfold, scoring='r2').mean()

0.69387675457406295

In [12]:
# Random forest
from sklearn.ensemble.forest import RandomForestRegressor
rfr = RandomForestRegressor() # TODO: pick features better? (cross-validation?)
rfr.fit(data, target)
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(rfr, data, target, cv=kfold, scoring='r2').mean()

0.7060468440360641

In [13]:
# SVM regression
from sklearn import svm
svm_reg = svm.SVR() # TODO: pick features better? (cross-validation?)
svm_reg.fit(data, target)
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(svm_reg, data, target, cv=kfold, scoring='r2').mean()

# LOL this is so fucked, why is that?
# plt.scatter(target, svm_reg.predict(data))
# plt.show()

-0.029355619675222756

In [14]:
# It appears that just throwing fancier models at it won't do the trick.
# Let's try to get better features selected before diving into the engineering.

In [15]:
# Univariate feature selection
pruned_data = fs.SelectKBest(fs.f_regression, k=1).fit_transform(data, target)

elastic_net = linear_model.ElasticNetCV(n_jobs=4, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=50000, normalize=True, cv=5)
elastic_net.fit(pruned_data, target)
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(elastic_net, pruned_data, target, cv=kfold, scoring='r2').mean()

0.70578716945728737

In [16]:
# God dammit, freaking just 1 variable (curr_eff) is enough to get 0.705865 R^2?
# Fuck, it's ALL in the engineering now. 

In [17]:
# Let's try PCA + ElasticNet
from sklearn import decomposition
pca = decomposition.PCA(n_components=25) # TODO: explicitly set n_components=???
pca_data = pca.fit_transform(data)

elastic_net = linear_model.ElasticNetCV(n_jobs=4, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=50000, normalize=True, cv=5)
elastic_net.fit(pca_data, target)
kfold = cross_validation.KFold(len(target), n_folds=5, shuffle=True)
cross_validation.cross_val_score(elastic_net, pca_data, target, cv=kfold, scoring='r2').mean()

0.73912177351336528

In [106]:
# Okay, even PCA didn't help. Let's make some features that take into account
# some trends, and then use ElasticNet.
# trend_df = pd.read_csv('../data/cleanedSeasonStats.csv')
trend_df = pd.read_csv('../data/test.csv')
for col in trend_df.columns:
    if 'Unnamed' in col or col[-1] == '-': # meaningless columns
        del trend_df[col]

# Scikit breaks with NaNs, replace with 0s
trend_df.fillna(0, inplace=True)

# For now, use everything numerical (except for obviously next_eff)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

# Clean the data more (remove outliers, redundancies)
illegal = {'pred', 'next-eff', 'next-eff-slope-1pb', 'next-eff-slope-2pb', 'next-eff-slope-3pb'}
noisy = {'year', 'per_poss-Rk', 'advanced-Rk'}
redundant = set(filter(lambda col: is_redundant(col), trend_df.columns.values)) | {'advanced-Age'}
trend_feature_cols = list(set(trend_df.select_dtypes(include=numerics).columns.values) \
                    - illegal - noisy - redundant)
print len(trend_df)
trend_df = trend_df[trend_df['per_poss-Age'] != 0]
trend_df = trend_df[trend_df['per_poss-TOV'] <= 20]
trend_df = trend_df[trend_df['per_poss-DRtg'] >= 80]
trend_df = trend_df[trend_df['advanced-ORB%'] <= 90]
trend_df = trend_df[trend_df['advanced-FTr'] < 1]
trend_df['sqrt-VORP'] = np.power(trend_df['advanced-VORP'], 2)
trend_df['log-GS'] = np.log(trend_df['per_poss-GS'] + 1)
trend_df['age-minutes'] = trend_df['per_poss-MP'] * trend_df['per_poss-Age']
trend_df['sqrt-GS'] = np.sqrt(trend_df['per_poss-GS'])

# trend_df = trend_df[trend_df['per_poss-2P%'] != 0]

# trend_df = trend_df[trend_df['per_poss-Pos'] == 'PF']
# ^ actually pretty helpful
# trend_df = trend_df[trend_df['year'] >= 1995]
# for col in trend_feature_cols + ['next-eff']:
#     trend_df = trend_df[trend_df[col] >= trend_df[col].quantile(.001)]
# trend_df = trend_df[trend_df['next-eff'] >= 0.1]
# trend_df = trend_df.iloc[np.random.permutation(len(trend_df))][:1600]
print len(trend_df)
trend_target = trend_df['next-eff'] 

# Get X, and scale/normalize (do we want/need to scale?)
trend_data = trend_df[trend_feature_cols]
trend_data = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(trend_data), columns=trend_data.columns)

# Train the model
# trend_elastic_net = RandomForestRegressor()
# trend_elastic_net = linear_model.LinearRegression()
# trend_elastic_net = linear_model.LassoCV(max_iter=5000, normalize=True, cv=5)
trend_elastic_net = linear_model.ElasticNetCV(n_jobs=4, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], max_iter=50000, normalize=True, cv=10)
# trend_elastic_net = svm.SVR(kernel='poly', C=1e3, degree=2)
trend_elastic_net.fit(trend_data, trend_target)

# Test performance of model
def percentage_error(model, X, y):
    return (abs(model.predict(X) - y)/abs(y)).mean()

kfold = cross_validation.KFold(len(trend_target), n_folds=10, shuffle=True)
cross_validation.cross_val_score(trend_elastic_net, trend_data, trend_target, cv=kfold, scoring='r2').mean()
# cross_validation.cross_val_score(trend_elastic_net, trend_data, trend_target, cv=kfold, scoring=percentage_error).mean()

10115
10041


0.74922708035901264

In [94]:
# plt.scatter(trend_target, trend_elastic_net.predict(trend_data))
# plt.show()

coefficients = sorted([(trend_feature_cols[i], trend_elastic_net.coef_[i]) for i in range(len(trend_feature_cols))], key=lambda tup: abs(tup[1]), reverse=True)
important_coef = filter(lambda tup: abs(tup[1]) != 0, coefficients)
important_coef

ValueError: coef_ is only available when using a linear kernel

In [ ]:
for tup in important_coef:
    if 'slope' in str(tup[0]):
        continue
    plt.figure()
    plt.title('{}'.format(str(tup[0]) + ' ' + str(tup[1])))
    plt.scatter(trend_df[tup[0]], trend_df['next-eff'])

In [ ]:
plt.scatter(trend_elastic_net.predict(trend_data), trend_target))

In [ ]:
# TODO: remove all outliers (bottom 10%?????)